In [2]:
import deepSI
from deepSI import System_data, System_data_list
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch import optim, nn
from tqdm.auto import tqdm
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
#%config InlineBackend.figure_formats = ['svg']

In [8]:
from deepSI.utils.torch_nets import CNN_encoder


#Psi(upast, ypast) where upast is an image and ypast an vector
#nb, nu, na, ny, nx
class CNN_encoder_image_input(nn.Module):
    def __init__(self, nb, nu, na, ny, nx, n_nodes_per_layer=64, n_hidden_layers=2, activation=nn.Tanh, features_ups_factor=1.33):
        super(CNN_encoder_image_input, self).__init__()
        self.net = CNN_encoder(na, ny, nb, nu, nx, \
                               n_nodes_per_layer=n_nodes_per_layer, \
                               n_hidden_layers=n_hidden_layers, \
                                activation=activation, \
                                features_ups_factor=features_ups_factor)

    def forward(self, upast, ypast):
        return self.net(ypast, upast)

#nx, nu
class CNN_encoder_f_image(nn.Module):
    def __init__(self, nx, nu, n_nodes_per_layer=64, n_hidden_layers=2, activation=nn.Tanh, features_ups_factor=1.33):
        super(CNN_encoder_f_image, self).__init__()
        self.net = CNN_encoder(1, nx, 1, nu, nx, \
                               n_nodes_per_layer=n_nodes_per_layer, \
                               n_hidden_layers=n_hidden_layers, \
                                activation=activation, \
                                features_ups_factor=features_ups_factor)

    def forward(self, x, u):
        #x.shape = (Nbatch, nx)
        #u.shape = (Nbatch, C, H, W)

        #self.net 
        # x -> upast = (Nbatch, nb, nu)
        # u -> ypast = (Nbatch, na, C, H, W)
        return self.net(x[:,None], u[:,None])
    
#nx, ny, nu=-1
class CNN_encoder_h_image(nn.Module):
    def __init__(self, nx, ny, nu, n_nodes_per_layer=64, n_hidden_layers=2, activation=nn.Tanh, features_ups_factor=1.33):
        super(CNN_encoder_h_image, self).__init__()
        self.net = CNN_encoder(1, nx, 1, nu, ny, \
                               n_nodes_per_layer=n_nodes_per_layer, \
                               n_hidden_layers=n_hidden_layers, \
                                activation=activation, \
                                features_ups_factor=features_ups_factor)

    def forward(self, x, u):
        #x.shape = (Nbatch, nx)
        #u.shape = (Nbatch, C, H, W)

        #self.net 
        # x -> upast = (Nbatch, nb, nu)
        # u -> ypast = (Nbatch, na, C, H, W)
        return self.net(x[:,None], u[:,None])



In [14]:
# from deepSI.utils import CNN_chained_upscales, CNN_encoder
from deepSI.fit_systems import SS_encoder_general

class SS_encoder_CNN_video_input(SS_encoder_general):
    """The subspace encoder convolutonal neural network with image inputs

    Notes
    -----
    The subspace encoder

    """
    def __init__(self, nx=10, na=20, nb=20, feedthrough=True, e_net=CNN_encoder_image_input, \
                 f_net=CNN_encoder_f_image, h_net=CNN_encoder_h_image, \
                                            e_net_kwargs={}, f_net_kwargs={}, h_net_kwargs={}):
        super(SS_encoder_CNN_video_input, self).__init__(nx=nx,na=na,nb=nb, feedthrough=feedthrough, \
            e_net=e_net,               f_net=f_net,                h_net=h_net, \
            e_net_kwargs=e_net_kwargs, f_net_kwargs=f_net_kwargs,  h_net_kwargs=h_net_kwargs)

model = SS_encoder_CNN_video_input(nx=3, na=50, nb=5)

nu = (3,25,30)
ny = 1
N = 200

u_seq = np.random.randn(N,*nu)
y_seq = np.random.randn(N, ny)

sys_data = System_data(u_seq, y_seq)

model.init_model(nu=(3,25,30), ny=1)


In [21]:
t = lambda x: torch.as_tensor(x, dtype=torch.float32)

upast, ypast, ufuture, yfuture = [t(x) for x in model.make_training_data(model.norm.transform(sys_data), nf=4)]

In [22]:
print(f'{upast.shape=}')
print(f'{ypast.shape=}')
print(f'{ufuture.shape=}')
print(f'{yfuture.shape=}')

xinit = model.encoder(upast, ypast)

upast.shape=torch.Size([147, 5, 3, 25, 30])
ypast.shape=torch.Size([147, 50, 1])
ufuture.shape=torch.Size([147, 4, 3, 25, 30])
yfuture.shape=torch.Size([147, 4, 1])


In [25]:
xnext = model.fn(xinit,ufuture[:,0])

In [26]:
ynext = model.hn(xnext,ufuture[:,1])


In [28]:
ynext.shape

torch.Size([147, 1])